In [1]:
pip install pysolr


     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.1 kB ? eta -:--:--
     -------------------------------- ----- 51.2/59.1 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 626.5 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pysolr: filename=pysolr-3.10.0-py2.py3-none-any.whl size=20095 sha256=d033f479f9464ef811389c504de7107334110603d2e923b4dc6696b3ab3c395b
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\b8\7f\b7\1526b5945096311831a68d98bb9ec

In [12]:
import pysolr

# Correct Solr URL pointing to the employee_data core
solr_url = 'http://localhost:8983/solr/employee_data'

# Initialize Solr connection
solr = pysolr.Solr(solr_url, always_commit=True)

# Try performing a search to test the connection
try:
    results = solr.search('*:*')  # Search for all documents
    for result in results:
        print(result)
except pysolr.SolrError as e:
    print(f"Error occurred: {e}")


{'Department': ['Engineering'], 'Gender': ['Male'], 'Ethnicity': ['Asian'], 'Age': [47], 'Country': ['United States'], 'City': ['Columbus'], 'id': '80d33121-4502-4533-8679-17d46524ed64', 'Employee_ID': ['E02002'], 'Full_Name': ['Kai Le'], 'Job_Title': ['Controls Engineer'], 'Business_Unit': ['Manufacturing'], 'Hire_Date': ['2/5/2022'], 'Annual_Salary': ['$92,368 '], 'Bonus__': ['0%'], '_version_': 1811884976682041344, '_root_': '80d33121-4502-4533-8679-17d46524ed64'}
{'Department': ['Sales'], 'Gender': ['Male'], 'Ethnicity': ['Asian'], 'Age': [58], 'Country': ['United States'], 'City': ['Chicago'], 'id': '9f9a163b-236b-4175-ab42-b45a726c917f', 'Employee_ID': ['E02003'], 'Full_Name': ['Robert Patel'], 'Job_Title': ['Analyst'], 'Business_Unit': ['Corporate'], 'Hire_Date': ['10/23/2013'], 'Annual_Salary': ['$45,703 '], 'Bonus__': ['0%'], '_version_': 1811884976739713024, '_root_': '9f9a163b-236b-4175-ab42-b45a726c917f'}
{'Department': ['IT'], 'Gender': ['Male'], 'Ethnicity': ['Asian'], 'A

In [22]:
import requests

def createCollection(p_collection_name):
    url = f'http://localhost:8983/solr/admin/cores?action=CREATE&name={p_collection_name}&configSet=_default'
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Collection {p_collection_name} created successfully.")
    else:
        print(f"Error creating collection {p_collection_name}: {response.text}")

In [16]:
def indexData(p_collection_name, p_exclude_column, data_file):
    # Load the CSV into a DataFrame
    df = pd.read_csv(data_file)
    
    # Exclude the specified column
    if p_exclude_column in df.columns:
        df = df.drop(columns=[p_exclude_column])
    
    # Convert the DataFrame to a dictionary and add it to Solr
    solr_url = f'http://localhost:8983/solr/{p_collection_name}'
    solr = pysolr.Solr(solr_url, always_commit=True)
    
    solr.add(df.to_dict(orient='records'))
    print(f"Data indexed into collection {p_collection_name}, excluding {p_exclude_column}.")


In [17]:
def searchByColumn(p_collection_name, p_column_name, p_column_value):
    solr_url = f'http://localhost:8983/solr/{p_collection_name}'
    solr = pysolr.Solr(solr_url, always_commit=True)
    
    # Build the query
    query = f'{p_column_name}:{p_column_value}'
    results = solr.search(query)
    
    print(f"Results for {p_column_name} = {p_column_value}:")
    for result in results:
        print(result)


In [18]:
def getEmpCount(p_collection_name):
    solr_url = f'http://localhost:8983/solr/{p_collection_name}'
    solr = pysolr.Solr(solr_url, always_commit=True)
    
    # Query to count all documents
    results = solr.search('*:*')
    
    print(f"Total employee count in collection {p_collection_name}: {results.hits}")


In [19]:
def delEmpById(p_collection_name, p_employee_id):
    solr_url = f'http://localhost:8983/solr/{p_collection_name}'
    solr = pysolr.Solr(solr_url, always_commit=True)
    
    solr.delete(id=p_employee_id)
    print(f"Deleted employee with ID {p_employee_id} from collection {p_collection_name}.")


In [20]:
def getDepFacet(p_collection_name):
    solr_url = f'http://localhost:8983/solr/{p_collection_name}'
    solr = pysolr.Solr(solr_url, always_commit=True)
    
    # Query with faceting by Department
    facet_query = {'q': '*:*', 'facet': 'true', 'facet.field': 'Department'}
    results = solr.search(q='*:*', **facet_query)
    
    # Print facet results
    facet_counts = results.facets['facet_fields']['Department']
    print("Employee count by Department:")
    for i in range(0, len(facet_counts), 2):
        print(f"{facet_counts[i]}: {facet_counts[i+1]}")


In [23]:
# Define collection names
v_nameCollection = 'collection_2'
v_phoneCollection = '15323'

# Create collections
createCollection(v_nameCollection)
createCollection(v_phoneCollection)

Collection collection_2 created successfully.
Collection 15323 created successfully.


In [24]:
# Get initial employee count
getEmpCount(v_nameCollection)

Total employee count in collection collection_2: 0


In [25]:
# Index data excluding Department for v_nameCollection and Gender for v_phoneCollection
indexData(v_nameCollection, 'Department', 'D:\\solr\\employee_data.csv')
indexData(v_phoneCollection, 'Gender', 'D:\\solr\\employee_data.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 1107: invalid start byte